# Parte 1: Consolidação de Medalhas por País

## Objetivos:
1. Obter total de medalhas por país (Verão, Inverno e Total Geral)
2. Criar tabelas ordenadas por total de medalhas
3. Gerar gráficos dos top 50 países

Resultados compatíveis com: https://pt.wikipedia.org/wiki/Quadro_de_medalhas_dos_Jogos_Ol%C3%ADmpicos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print(f"pandas: {pd.__version__}")
print(f"matplotlib: {plt.matplotlib.__version__}")

## 1. Carregar Dados da Camada Bronze

In [ ]:
# Definir paths
BASE_PATH = Path('..')
BRONZE_PATH = BASE_PATH / 'bronze'
GOLD_PATH = BASE_PATH / 'gold'
OUTPUT_PATH = BASE_PATH / 'outputs'

# Criar diretórios se não existirem
GOLD_PATH.mkdir(exist_ok=True)
(OUTPUT_PATH / 'tables').mkdir(parents=True, exist_ok=True)
(OUTPUT_PATH / 'figures').mkdir(parents=True, exist_ok=True)

print("✓ Paths configurados")

In [ ]:
# Carregar dados
print("Carregando dados da camada Bronze...")

df_medal_tally = pd.read_parquet(BRONZE_PATH / 'medal_tally.parquet')
df_game = pd.read_parquet(BRONZE_PATH / 'game.parquet')
df_country = pd.read_parquet(BRONZE_PATH / 'country.parquet')

print(f"✓ medal_tally: {df_medal_tally.shape}")
print(f"✓ game: {df_game.shape}")
print(f"✓ country: {df_country.shape}")

In [ ]:
# Explorar dados
print("\nPrimeiras linhas de medal_tally:")
display(df_medal_tally.head())

print("\nInformação de temporadas:")
print(df_medal_tally['season'].value_counts())

## 2. Consolidação de Medalhas

### 2.1 Medalhas - Jogos de Verão

In [ ]:
print("\n" + "="*80)
print("MEDALHAS - JOGOS DE VERÃO")
print("="*80)

# Filtrar apenas jogos de verão
summer_medals = df_medal_tally[df_medal_tally['season'] == 'Summer'].copy()

# Agregar por país
medals_summer = summer_medals.groupby(['country', 'country_noc']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index()

# Ordenar por total de medalhas
medals_summer = medals_summer.sort_values('total', ascending=False).reset_index(drop=True)
medals_summer['rank'] = range(1, len(medals_summer) + 1)

# Reordenar colunas
medals_summer = medals_summer[['rank', 'country', 'country_noc', 'gold', 'silver', 'bronze', 'total']]

print(f"\nTotal de países: {len(medals_summer)}")
print(f"Total de medalhas distribuídas: {medals_summer['total'].sum()}")
print(f"\nTop 10 países:")
display(medals_summer.head(10))

### 2.2 Medalhas - Jogos de Inverno

In [ ]:
print("\n" + "="*80)
print("MEDALHAS - JOGOS DE INVERNO")
print("="*80)

# Filtrar apenas jogos de inverno
winter_medals = df_medal_tally[df_medal_tally['season'] == 'Winter'].copy()

# Agregar por país
medals_winter = winter_medals.groupby(['country', 'country_noc']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index()

# Ordenar por total de medalhas
medals_winter = medals_winter.sort_values('total', ascending=False).reset_index(drop=True)
medals_winter['rank'] = range(1, len(medals_winter) + 1)

# Reordenar colunas
medals_winter = medals_winter[['rank', 'country', 'country_noc', 'gold', 'silver', 'bronze', 'total']]

print(f"\nTotal de países: {len(medals_winter)}")
print(f"Total de medalhas distribuídas: {medals_winter['total'].sum()}")
print(f"\nTop 10 países:")
display(medals_winter.head(10))

### 2.3 Medalhas - Total Geral

In [ ]:
print("\n" + "="*80)
print("MEDALHAS - TOTAL GERAL")
print("="*80)

# Agregar todas as medalhas
medals_total = df_medal_tally.groupby(['country', 'country_noc']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index()

# Ordenar por total de medalhas
medals_total = medals_total.sort_values('total', ascending=False).reset_index(drop=True)
medals_total['rank'] = range(1, len(medals_total) + 1)

# Reordenar colunas
medals_total = medals_total[['rank', 'country', 'country_noc', 'gold', 'silver', 'bronze', 'total']]

print(f"\nTotal de países: {len(medals_total)}")
print(f"Total de medalhas distribuídas: {medals_total['total'].sum()}")
print(f"\nTop 10 países:")
display(medals_total.head(10))

## 3. Salvar Resultados na Camada Gold

In [ ]:
def save_with_metadata(df, filename, description):
    """Salva DataFrame em parquet com metadados"""
    # Salvar parquet
    filepath = GOLD_PATH / f"{filename}.parquet"
    df.to_parquet(filepath, index=False)
    print(f"✓ Salvo: {filepath}")
    
    # Criar metadados
    metadata = {
        "dataset_name": filename,
        "description": description,
        "created_at": datetime.now().isoformat(),
        "source": "Consolidado de medal_tally (bronze layer)",
        "shape": {"rows": len(df), "columns": len(df.columns)},
        "columns": list(df.columns),
        "dtypes": df.dtypes.astype(str).to_dict()
    }
    
    # Salvar metadados
    metadata_path = GOLD_PATH / f"{filename}_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    print(f"✓ Metadados salvos: {metadata_path}")
    
    # Salvar também como CSV para fácil visualização
    csv_path = OUTPUT_PATH / 'tables' / f"{filename}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"✓ CSV salvo: {csv_path}")

print("\nSalvando resultados...\n")

save_with_metadata(medals_summer, 'medals_summer_consolidated', 
                   'Total de medalhas por país nos Jogos de Verão (1896-2022)')
print()
save_with_metadata(medals_winter, 'medals_winter_consolidated',
                   'Total de medalhas por país nos Jogos de Inverno (1924-2022)')
print()
save_with_metadata(medals_total, 'medals_total_consolidated',
                   'Total geral de medalhas por país (Verão + Inverno, 1896-2022)')

## 4. Visualizações - Top 50 Países

### 4.1 Gráfico - Jogos de Verão (Top 50)

In [ ]:
# Top 50 países - Verão
top50_summer = medals_summer.head(50).copy()

# Preparar dados para gráfico empilhado
fig, ax = plt.subplots(figsize=(16, 12))

y_pos = np.arange(len(top50_summer))
width = 0.8

# Criar barras empilhadas (de baixo para cima: bronze, prata, ouro)
p1 = ax.barh(y_pos, top50_summer['bronze'], width, label='Bronze', color='#CD7F32')
p2 = ax.barh(y_pos, top50_summer['silver'], width, left=top50_summer['bronze'], 
             label='Prata', color='#C0C0C0')
p3 = ax.barh(y_pos, top50_summer['gold'], width, 
             left=top50_summer['bronze'] + top50_summer['silver'], 
             label='Ouro', color='#FFD700')

# Configurar eixos
ax.set_yticks(y_pos)
ax.set_yticklabels(top50_summer['country_noc'])
ax.invert_yaxis()  # Top no topo
ax.set_xlabel('Total de Medalhas', fontsize=12, fontweight='bold')
ax.set_ylabel('País', fontsize=12, fontweight='bold')
ax.set_title('Top 50 Países - Medalhas nos Jogos de Verão (1896-2022)', 
             fontsize=14, fontweight='bold', pad=20)

# Adicionar valores totais nas barras
for i, (idx, row) in enumerate(top50_summer.iterrows()):
    ax.text(row['total'] + 20, i, str(int(row['total'])), 
            va='center', fontweight='bold', fontsize=9)

ax.legend(loc='lower right', fontsize=10)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()

# Salvar
output_file = OUTPUT_PATH / 'figures' / 'top50_medals_summer.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✓ Gráfico salvo: {output_file}")
plt.show()

### 4.2 Gráfico - Jogos de Inverno (Top 50)

In [ ]:
# Top 50 países - Inverno (ou todos se for menos de 50)
n_countries = min(50, len(medals_winter))
top50_winter = medals_winter.head(n_countries).copy()

# Preparar dados para gráfico empilhado
fig, ax = plt.subplots(figsize=(16, 12))

y_pos = np.arange(len(top50_winter))
width = 0.8

# Criar barras empilhadas
p1 = ax.barh(y_pos, top50_winter['bronze'], width, label='Bronze', color='#CD7F32')
p2 = ax.barh(y_pos, top50_winter['silver'], width, left=top50_winter['bronze'], 
             label='Prata', color='#C0C0C0')
p3 = ax.barh(y_pos, top50_winter['gold'], width, 
             left=top50_winter['bronze'] + top50_winter['silver'], 
             label='Ouro', color='#FFD700')

# Configurar eixos
ax.set_yticks(y_pos)
ax.set_yticklabels(top50_winter['country_noc'])
ax.invert_yaxis()
ax.set_xlabel('Total de Medalhas', fontsize=12, fontweight='bold')
ax.set_ylabel('País', fontsize=12, fontweight='bold')
ax.set_title(f'Top {n_countries} Países - Medalhas nos Jogos de Inverno (1924-2022)', 
             fontsize=14, fontweight='bold', pad=20)

# Adicionar valores totais
for i, (idx, row) in enumerate(top50_winter.iterrows()):
    ax.text(row['total'] + 5, i, str(int(row['total'])), 
            va='center', fontweight='bold', fontsize=9)

ax.legend(loc='lower right', fontsize=10)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()

# Salvar
output_file = OUTPUT_PATH / 'figures' / 'top50_medals_winter.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✓ Gráfico salvo: {output_file}")
plt.show()

### 4.3 Gráfico - Total Geral (Top 50)

In [ ]:
# Top 50 países - Total
top50_total = medals_total.head(50).copy()

# Preparar dados para gráfico empilhado
fig, ax = plt.subplots(figsize=(16, 12))

y_pos = np.arange(len(top50_total))
width = 0.8

# Criar barras empilhadas
p1 = ax.barh(y_pos, top50_total['bronze'], width, label='Bronze', color='#CD7F32')
p2 = ax.barh(y_pos, top50_total['silver'], width, left=top50_total['bronze'], 
             label='Prata', color='#C0C0C0')
p3 = ax.barh(y_pos, top50_total['gold'], width, 
             left=top50_total['bronze'] + top50_total['silver'], 
             label='Ouro', color='#FFD700')

# Configurar eixos
ax.set_yticks(y_pos)
ax.set_yticklabels(top50_total['country_noc'])
ax.invert_yaxis()
ax.set_xlabel('Total de Medalhas', fontsize=12, fontweight='bold')
ax.set_ylabel('País', fontsize=12, fontweight='bold')
ax.set_title('Top 50 Países - Total Geral de Medalhas (Verão + Inverno, 1896-2022)', 
             fontsize=14, fontweight='bold', pad=20)

# Adicionar valores totais
for i, (idx, row) in enumerate(top50_total.iterrows()):
    ax.text(row['total'] + 30, i, str(int(row['total'])), 
            va='center', fontweight='bold', fontsize=9)

ax.legend(loc='lower right', fontsize=10)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()

# Salvar
output_file = OUTPUT_PATH / 'figures' / 'top50_medals_total.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✓ Gráfico salvo: {output_file}")
plt.show()

## 5. Estatísticas Resumidas

In [ ]:
print("\n" + "="*80)
print("ESTATÍSTICAS RESUMIDAS - PARTE 1")
print("="*80)

print("\n1. JOGOS DE VERÃO")
print(f"   Países participantes: {len(medals_summer)}")
print(f"   Total de medalhas: {medals_summer['total'].sum()}")
print(f"   Média de medalhas por país: {medals_summer['total'].mean():.2f}")
print(f"   Top 3 países:")
for i, row in medals_summer.head(3).iterrows():
    print(f"      {row['rank']}º {row['country']} ({row['country_noc']}): {int(row['total'])} medalhas")

print("\n2. JOGOS DE INVERNO")
print(f"   Países participantes: {len(medals_winter)}")
print(f"   Total de medalhas: {medals_winter['total'].sum()}")
print(f"   Média de medalhas por país: {medals_winter['total'].mean():.2f}")
print(f"   Top 3 países:")
for i, row in medals_winter.head(3).iterrows():
    print(f"      {row['rank']}º {row['country']} ({row['country_noc']}): {int(row['total'])} medalhas")

print("\n3. TOTAL GERAL")
print(f"   Países participantes: {len(medals_total)}")
print(f"   Total de medalhas: {medals_total['total'].sum()}")
print(f"   Média de medalhas por país: {medals_total['total'].mean():.2f}")
print(f"   Top 3 países:")
for i, row in medals_total.head(3).iterrows():
    print(f"      {row['rank']}º {row['country']} ({row['country_noc']}): {int(row['total'])} medalhas")

print("\n" + "="*80)
print("✓ PARTE 1 CONCLUÍDA COM SUCESSO!")
print("="*80)